In [1]:
!pip install googletrans
%pip install translate
!pip install validators
!pip install deep_translator



  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15715 sha256=ad7e58dd6a7ddaeb58b506431d1d1f459203ca8371f83253df082d0bf15c3aa5
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling cha

In [2]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
import nltk
from nltk.tokenize import word_tokenize
import re
from deep_translator import GoogleTranslator

# Download the punkt tokenizer if not already downloaded
nltk.download('punkt')

# Load the delicate text detection model
delicate_text_classifier = pipeline("text-classification", model="grammarly/detexd-roberta-base")

def predict_binary_score(text: str):
    # Get multiclass probability scores
    scores = delicate_text_classifier(text, top_k=None)

    # Convert to a single score by summing the probability scores
    # for the higher-index classes
    return sum(score['score'] for score in scores if score['label'] in ('LABEL_3', 'LABEL_4', 'LABEL_5'))

def predict_delicate(text: str, threshold=0.5):
    return predict_binary_score(text) > threshold

def extract_delicate_text(lines):
    # Extract and return delicate text for each line
    return [(original, translated) for original, translated in zip(lines, [translate_to_english(line, "auto") for line in lines]) if predict_delicate(translated)]

def remove_trailing_uppercase(line):
    # Remove trailing uppercase letter if the line ends with one
    if line and line[-1].isupper():
        line = line[:-1]
    return line

# Function to translate text to English using deep_translator
def translate_to_english(text, source_lang='auto'):
    translator = GoogleTranslator(source=source_lang, target='en')
    translation = translator.translate(text)
    return translation

def analyze_website_delicate_text(url):
    try:
        # Fetch web content
        response = requests.get(url)
        response.raise_for_status()

        # Extract all visible text from HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        text_content = soup.get_text()

        # Split the text by newline ("\n") or "." characters using regex and save to array
        regex = r"[\n.]"
        lines = [line.strip() for line in re.split(regex, text_content) if line.strip()]

        # Remove lines with words of length less than 1
        lines = [line for line in lines if all(len(word) >= 1 for word in word_tokenize(line))]

        # Remove trailing uppercase letter if the line ends with one
        lines = [remove_trailing_uppercase(line) for line in lines]
        #print(lines)

        # Extract and print delicate text for each line
        delicate_text = extract_delicate_text(lines)

        # Print delicate text in the original language
        print("\nDelicate text:")
        for original, translated in delicate_text:
            print("-", original)

        # Calculate the percentage of delicate text
        total_words = len(lines)
        delicate_percentage = (len(delicate_text) / total_words) * 100
        print(f"\nPercentage of delicate text: {delicate_percentage:.2f}%")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching the website content: {e}")

# Prompt user to enter a URL
website_url = input("Enter the URL of the website: ")

# Example usage
analyze_website_delicate_text(website_url)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Enter the URL of the website: https://github.com/Pyroflica/Proposal_Report/edit/main/README.md

Delicate text:

Percentage of delicate text: 0.00%
